In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [5]:
dataset = os.listdir('Downloads/dataset')

In [6]:
page = pd.read_csv('Downloads/dataset/TBN_CUST_BEHAVIOR.csv')
txn = pd.read_csv('Downloads/dataset/TBN_WM_TXN.csv')
identity = pd.read_csv('Downloads/dataset/TBN_CIF.csv')
dt = pd.read_csv('Downloads/dataset/TBN_RECENT_DT.csv')
loan = pd.read_csv('Downloads/dataset/TBN_LN_APPLY.csv')
fx = pd.read_csv('Downloads/dataset/TBN_FX_TXN.csv')
card = pd.read_csv('Downloads/dataset/TBN_CC_APPLY.csv')

In [7]:
txn.rename(columns = {'TXN_DT': 'I_TXN_DT', }, inplace = True)
card.rename(columns = {'TXN_DT': 'C_TXN_DT', }, inplace = True)
fx.rename(columns = {'TXN_DT': 'F_TXN_DT', }, inplace = True)
loan.rename(columns = {'TXN_DT': 'L_TXN_DT', }, inplace = True)

In [5]:
df_dataset = {'網頁瀏覽':page, '信託交易':txn, '基本屬性':identity, '信用卡':card, 
              '外匯交易':fx, '信貸申請':loan, '最後交易':dt}

In [6]:
for i in df_dataset:
    print(i, df_dataset[i].shape, 'unique_customer = {}'.format(len(df_dataset[i]['CUST_NO'].drop_duplicates())), 
          '\n', list(df_dataset[i].columns), end='\n\n')

網頁瀏覽 (2209864, 3) unique_customer = 195000 
 ['CUST_NO', 'VISITDATE', 'PAGE']

信託交易 (36763, 5) unique_customer = 14004 
 ['CUST_NO', 'I_TXN_DT', 'CUST_RISK_CODE', 'INVEST_TYPE_CODE', 'WM_TXN_AMT']

基本屬性 (187679, 8) unique_customer = 187679 
 ['CUST_NO', 'AGE', 'CHILDREN_CNT', 'CUST_START_DT', 'EDU_CODE', 'GENDER_CODE', 'INCOME_RANGE_CODE', 'WORK_MTHS']

信用卡 (54393, 2) unique_customer = 44112 
 ['CUST_NO', 'C_TXN_DT']

外匯交易 (507185, 3) unique_customer = 74111 
 ['CUST_NO', 'F_TXN_DT', 'FX_TXN_AMT']

信貸申請 (6741, 4) unique_customer = 6654 
 ['CUST_NO', 'L_TXN_DT', 'LN_AMT', 'LN_USE']

最後交易 (195000, 5) unique_customer = 195000 
 ['CUST_NO', 'CC_RECENT_DT', 'FX_RECENT_DT', 'LN_RECENT_DT', 'WM_RECENT_DT']



In [8]:
# 編輯資料
txn['NEW_TXN_DT'] = txn['I_TXN_DT'].apply(lambda x:x-9447)
card['NEW_TXN_DT'] = card['C_TXN_DT'].apply(lambda x:x-9447)
fx['NEW_TXN_DT'] = fx['F_TXN_DT'].apply(lambda x:x-9447)
loan['NEW_TXN_DT'] = loan['L_TXN_DT'].apply(lambda x:x-9447)

# CUST_BEHAVIOR_PATH

In [13]:
data = pd.read_csv('Downloads/TBN_CUST_BEHAVIOR_PATH.csv')

In [14]:
# 對瀏覽行為加工
page['NEW_VISITDATE'] = page['VISITDATE'].apply(lambda x:x-9447)
def add(x):
    path1 = x[0]
    path2 = x[1]
    return '/'.join([x[0],x[1]])
data['new_path'] = data[['PATH1', 'PATH2']].apply(add, axis=1)
d_page = data[['CUST_NO', 'VISITDATE', 'new_path']]
d_page['NEW_VISITDATE'] = d_page['VISITDATE'].apply(lambda x:x-9447)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
d_page.head()

,CUST_NO,VISITDATE,new_path,NEW_VISITDATE
0,---CHVW7DUN8SZLO,9472,edrn/pgusordq,25
1,---CHVW7DUN8SZLO,9472,gygrt/cugfkt,25
2,---CHVW7DUN8SZLO,9499,edrn/deoxt,52
3,---CHVW7DUN8SZLO,9499,edrn/pgusordq,52
4,---CHVW7DUN8SZLO,9499,edrn/pgusordq,52


# txn x page

In [179]:
# test
page_1_30 = selector(d_page, 'NEW_VISITDATE', 1, 30)
txn_31_60 = selector(txn, 'NEW_TXN_DT', 30, 60)
txn_page_1_60 = pd.merge(page_1_30, 
                    txn_31_60, on='CUST_NO', how='left')[['CUST_NO', 'new_path', 'NEW_VISITDATE', 'NEW_TXN_DT']]

In [180]:
txn_page_1_60 .head()

,CUST_NO,new_path,NEW_VISITDATE,NEW_TXN_DT
0,---CHVW7DUN8SZLO,edrn/pgusordq,25,NaN
1,---CHVW7DUN8SZLO,gygrt/cugfkt,25,NaN
2,--1YNJZSSOXDF6UU,edrn/deoxt,2,NaN
3,--1YNJZSSOXDF6UU,gygrt/mgmegutgdm,2,NaN
4,--1YNJZSSOXDF6UU,gygrt/pke,17,NaN


In [460]:
unique_page = cmm(txn_page_1_60,'CUST_NO', 'NEW_VISITDATE')
unique_txn = cmm(txn_page_1_60,'CUST_NO', 'NEW_TXN_DT')

In [461]:
# MultiLabelBinarizerone_hot = MultiLabelBinarizer()
path_m = txn_page_1_60[['CUST_NO', 'new_path']].groupby('CUST_NO')['new_path'].apply(lambda x: "%s" % ','.join(x))
df_path_m = pd.DataFrame(path_m)
df_path_m['new_path'] = df_path_m['new_path'].apply(c)
path_encod = one_hot.fit_transform(df_path_m['new_path'])
unique_cust = txn_page['CUST_NO'].drop_duplicates().reset_index().drop('index', axis=1)

In [462]:
c_path = pd.concat([unique_cust, pd.DataFrame(path_encod, columns=one_hot.classes_)], axis=1)
d_all = pd.merge(pd.merge(c_path, unique_page), unique_txn)

In [465]:
d_all['y'] = d_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
d_all_x = d_all[list(d_all.columns[1:-4])]
d_all_y = d_all['y']

# functions

In [176]:
def selector(df, col, min_date, max_date):
    return df[(df[col]>=min_date) & (df[col]<=max_date)]
def cmm(d, key, variable):
    d = d[[key, variable]].groupby(key)
    d_m = pd.merge(pd.merge(d.count(), d.min(), left_index=True, right_index=True), 
               d.max(), left_index=True, right_index=True)
    d_m.rename(columns = {'{}_x'.format(variable): '{}_Count'.format(variable), 
                          '{}_y'.format(variable): '{}_Min'.format(variable), 
                          '{}'.format(variable): '{}_Max'.format(variable)}, inplace = True)
    return d_m.reset_index()
def c(x):
    return x.split(',')
def merge_path(d):
    path_m = d[['CUST_NO', 'new_path']].groupby('CUST_NO')['new_path'].apply(lambda x: "%s" % ','.join(x))
    df_path_m = pd.DataFrame(path_m)
    df_path_m['new_path'] = df_path_m['new_path'].apply(c)
    return df_path_m
# MultiLabelBinarizer
def encoding(d, unique_cust):
    from sklearn.preprocessing import MultiLabelBinarizer
    one_hot = MultiLabelBinarizer()
    path_encod = one_hot.fit_transform(d['new_path'])
    c_path = pd.concat([unique_cust, pd.DataFrame(path_encod, columns=one_hot.classes_)], axis=1)
    return c_path

In [177]:
def f1_score_1(str_report):
    return float(list(filter(lambda x:x!='', str_report.split('\n')[3].split(' ')))[3])
def decicionTest(tra_X, tra_Y, tes_X, tes_Y):
    dtree = DecisionTreeClassifier()
    dtree.fit(tra_X,tra_Y)
    predictions = dtree.predict(tes_X)
    return predictions, classification_report(tes_Y,predictions), confusion_matrix(tes_Y,predictions)
def max_score_day(m_score):
    max_score = max(m_score.values())
    max_days = [i for i in m_score.keys() if m_score[i] == max(m_score.values())]
    avg_max_days = sum(max_days)/len(max_days)
    return max_score, avg_max_days

# N x txn x page

In [182]:
selector(d_page, 'NEW_VISITDATE', 26, 30).head()

,CUST_NO,VISITDATE,new_path,NEW_VISITDATE
42,--8BVNAAVAUSW6X8,9473,edrn/pgusordq,26
46,--9QPGEBDTE8XEWM,9473,gygrt/se,26
51,--BC4RZRS03GIM2W,9473,edrn/deoxt,26
61,--DQ9OSIQ6MMBHD0,9475,edrn/deoxt,28
89,--OCXKMXUHPJUDHE,9476,edrn/pgusordq,29


In [46]:
class M(object):
#     交易從第f天開始，持續30天，瀏覽從第f-m天開始，持續m天期間
    def __init__(self, first_txn_day, m_size=30, first_view_day=None):
        self.first_txn_day = first_txn_day
        self.m_size = m_size
        self.first_view_day = first_txn_day - m_size
#     ＋m天後的交易
    def t_p(self):
        txn_ = selector(txn, 'NEW_TXN_DT', self.first_txn_day, (self.first_txn_day+29))
        page_ = selector(d_page, 'NEW_VISITDATE', self.first_view_day, (self.first_txn_day-1))
        txn_page = pd.merge(page_, txn_, on='CUST_NO', how='left')[['CUST_NO', 
                                                                    'new_path', 'NEW_VISITDATE', 'NEW_TXN_DT']]
        return txn_page
#     產生屬性
    def generate(self):
        self.txn_page = self.t_p()
        self.unique_page = cmm(self.txn_page,'CUST_NO', 'NEW_VISITDATE')
        self.unique_txn = cmm(self.txn_page,'CUST_NO', 'NEW_TXN_DT')
        self.merge_path = merge_path(self.txn_page)
        self.unique_cust = self.txn_page['CUST_NO'].drop_duplicates().reset_index().drop('index', axis=1)
        self.c_path = encoding(self.merge_path, self.unique_cust)
        self.all = pd.merge(pd.merge(self.c_path, self.unique_page), self.unique_txn)
        self.all['y'] = self.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
        self.all_x = self.all[list(self.all.columns[1:-4])]
        self.all_y = self.all['y']
    def check(self):
        words = '''first_view_day: {}, m_size: {}, first_txn_day: {}'''
        return words.format(self.first_view_day, self.m_size, self.first_txn_day)

In [184]:
t_31 = M(31, 5)
t_31.generate()
t_31.check()

'first_view_day: 26, m_size: 5, first_txn_day: 31'

In [186]:
t_61 = M(61,5)
t_61.generate()
t_61.check()

'first_view_day: 56, m_size: 5, first_txn_day: 61'

In [194]:
print(t_31.all.shape, t_61.all.shape)

(22630, 49) (24133, 48)


In [187]:
intersect = list(set(list(t_31.all_x.columns)) & set(list(t_61.all_x.columns)))
t_31.all_x = t_31.all_x[intersect]
t_61.all_x = t_61.all_x[intersect]

In [188]:
dtree = DecisionTreeClassifier()
dtree.fit(t_31.all_x,t_31.all_y)
predictions = dtree.predict(t_61.all_x)

In [192]:
print(f1_score_1(classification_report(t_61.all_y,predictions)))

0.16


# alter time series

In [195]:
t_66 = M(66,5)
t_66.generate()
t_66.check()

'first_view_day: 61, m_size: 5, first_txn_day: 66'

In [197]:
intersect = list(set(list(t_31.all_x.columns)) & set(list(t_61.all_x.columns)) & set(list(t_66.all_x.columns)))
t_31.all_x = t_31.all_x[intersect]
t_61.all_x = t_61.all_x[intersect]
t_66.all_x = t_66.all_x[intersect]

In [203]:
t_61.all_x.shape

(24133, 41)

In [204]:
pd.concat([t_61.all_x,t_66.all_x]).shape

(46971, 41)

In [205]:
dtree = DecisionTreeClassifier()
dtree.fit(t_31.all_x,t_31.all_y)
predictions = dtree.predict(pd.concat([t_61.all_x,t_66.all_x]))

In [207]:
print(f1_score_1(classification_report(pd.concat([t_61.all_y,t_66.all_y]),predictions)))

0.18


In [235]:
d = [[1,2,3,4], [2,3,4], [3,4,5,6,7]]
set(d[0]).intersection(*d)

{3, 4}

In [243]:
trains = []
for i in range(31,1,-5):
    train = M(i,5)
    train.generate()
    train.check()
    trains.append(train)

In [244]:
tests = []
for i in range(61,31,-5):
    test = M(i,5)
    test.generate()
    test.check()
    tests.append(test)

In [252]:
d = [list(i.all_x.columns) for i in trains+tests]
intersect = list(set(d[0]).intersection(*d))
print(len(intersect))

37


In [254]:
for i in trains:
    i.all_x = i.all_x[intersect]
for i in tests:
    i.all_x = i.all_x[intersect]

In [264]:
dtree = DecisionTreeClassifier()
dtree.fit(pd.concat([i.all_x for i in trains]),pd.concat([i.all_y for i in trains]))
predictions = dtree.predict(pd.concat([i.all_x for i in tests]))

In [266]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [267]:
print(f1_score_1(classification_report(pd.concat([i.all_y for i in tests]),predictions)))

0.17


# train / test

In [44]:
def comparing_score(train_start_day, test_start_day, m_size):
    train = M(train_start_day, m_size)
    train.generate()
    test = M(test_start_day, m_size)
    test.generate()
#     info = 'train - {}\ntest - {}'.format(train.check(), test.check())
    intersect = list(set(list(train.all_x.columns)) & set(list(test.all_x.columns)))
    train.all_x = train.all_x[intersect]
    test.all_x = test.all_x[intersect]
    predictions, report, matrix = decicionTest(train.all_x, train.all_y, test.all_x, test.all_y)
    score = f1_score_1(report)
    return score, [train, test, predictions, report, matrix]

In [47]:
f_score, f_ref = comparing_score(31, 61, 14)
print(f_score)

0.15


In [49]:
# different m size for score
m_score = {}
for i in range(5,15):
    m_score[i] = comparing_score(31, 61, i)[1]
    print(i, end=' ')
#     print('m={}, score={}'.format(i, comparing_score(31, 61, i)[1]))

5 6 7 8 9 10 11 12 13 14 

In [88]:
max_score_day(m_score)

(0.17, 5.0)

In [71]:
for i in range(61,30,-1):
    print(i, end=' ')

61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 

In [75]:
d_record = {}
for train_start_day in range(61,30,-1):
    print('train_start_day: {}'.format(train_start_day))
    m_score = {}
    for m in range(1,15):
        m_score[m] = comparing_score(train_start_day=train_start_day, test_start_day=91, m_size=m)[1]
        print(m, end=' ')    
    d_record[train_start_day] = m_score
    print('\n')

train_start_day: 61
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 60
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 59
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 58
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 57
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 56
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 55
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 54
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 53
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 52
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 51
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 50
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 49
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 48
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 47
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 46
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 45
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 44
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_star

In [89]:
max_d_record = {}
for k in list(d_record.keys()):
    max_d_record[k] = max_score_day(d_record[k])

In [43]:
max_d_record.values()

NameError: name 'max_d_record' is not defined

# multiple train / test

In [271]:
def x_versus_page(p, first_txn_day, x, first_view_day):
    x_ = selector(x, 'NEW_TXN_DT', first_txn_day, (first_txn_day+29))
    page_ = selector(p, 'NEW_VISITDATE', first_view_day, (first_txn_day-1))
    x_page = pd.merge(page_, x_, on='CUST_NO', how='left')[['CUST_NO', 
                                                                'new_path', 'NEW_VISITDATE', 'NEW_TXN_DT']]
    return x_page

In [457]:
class X(object):
    def __init__(self, first_txn_day, m_size=30, first_view_day=None):
        self.first_txn_day = first_txn_day
        self.m_size = m_size
        self.first_view_day = first_txn_day - m_size
#     基本資料
    def attach(self):
        self.p=d_page
        self.t=txn
        self.c=card
        self.f=fx
        self.l=loan
#     產生屬性
    def generate_1(self, x):
        self.x_page = x_versus_page(self.p, self.first_txn_day, x, self.first_view_day)
        self.unique_page = cmm(self.x_page,'CUST_NO', 'NEW_VISITDATE')
        self.unique_txn = cmm(self.x_page,'CUST_NO', 'NEW_TXN_DT')
        self.merge_path = merge_path(self.x_page)
        self.unique_cust = self.x_page['CUST_NO'].drop_duplicates().reset_index().drop('index', axis=1)
        self.c_path = encoding(self.merge_path, self.unique_cust)
        self.all = pd.merge(pd.merge(self.c_path, self.unique_page), self.unique_txn)
    def generate_2(self):
        self.all = self.all.reindex(sorted(self.all.columns), axis=1)
        self.all['y'] = self.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
        self.all_x = self.all[list(self.all.columns[4:-1])]
        self.all_y = self.all['y']

In [357]:
x_versus_page(d_page, 31, txn, 26).head()

,CUST_NO,new_path,NEW_VISITDATE,NEW_TXN_DT
0,--8BVNAAVAUSW6X8,edrn/pgusordq,26,NaN
1,--9QPGEBDTE8XEWM,gygrt/se,26,NaN
2,--BC4RZRS03GIM2W,edrn/deoxt,26,NaN
3,--DQ9OSIQ6MMBHD0,edrn/deoxt,28,NaN
4,--OCXKMXUHPJUDHE,edrn/pgusordq,29,NaN


In [358]:
c_31 = X(31, 14)
c_31.attach()
c_31.generate_1(c_31.c)
c_31.generate_2()

In [359]:
c_31.all.head()

,CUST_NO,edrn/coupoudtg,edrn/deoxt,edrn/fkjktdq,edrn/guuou.dspz,edrn/pgusordq,edrn/rgwmoekqg,edrn/sci-cortdct-mg,edrn/smdqq-exskrgss,fgmo/wgedtm,...,s/seqodr,s/wgdqthcdqcxqdtkor,wgdqth/joqfpdsseoon,NEW_VISITDATE_Count,NEW_VISITDATE_Min,NEW_VISITDATE_Max,NEW_TXN_DT_Count,NEW_TXN_DT_Min,NEW_TXN_DT_Max,y
0,---CHVW7DUN8SZLO,0,0,0,0,1,0,0,0,0,...,0,0,0,2,25,25,0,NaN,NaN,0
1,--1YNJZSSOXDF6UU,0,0,0,0,1,0,0,0,0,...,0,0,0,3,17,18,0,NaN,NaN,0
2,--3NF5SUEEEAWOPI,0,0,0,0,1,0,0,0,0,...,0,0,0,1,22,22,0,NaN,NaN,0
3,--8BVNAAVAUSW6X8,0,0,0,0,1,0,0,0,0,...,0,0,0,1,26,26,0,NaN,NaN,0
4,--9QPGEBDTE8XEWM,0,0,0,0,0,0,0,0,0,...,0,0,0,1,26,26,0,NaN,NaN,0


In [455]:
def x_comparing_score(train_start_day, test_start_day, m_size, tar):
#     建立物件
    train = X(train_start_day, m_size)
    test = X(test_start_day, m_size)
    train.attach()
    test.attach()
#     選擇目標
    if tar=='t':
        train.generate_1(train.t)
        test.generate_1(test.t)
        train.generate_2()
        test.generate_2()
    elif tar=='c':
        train.generate_1(train.c)
        test.generate_1(test.c)
        train.generate_2()
        test.generate_2()
    elif tar=='f':
        train.generate_1(train.f)
        test.generate_1(test.f)
        train.generate_2()
        test.generate_2()
    elif tar=='l':
        train.generate_1(train.l)
        test.generate_1(test.l)
        train.generate_2()
        test.generate_2()
#     篩選交集
    intersect = list(set(list(train.all_x.columns)) & set(list(test.all_x.columns)))
    train.all_x = train.all_x[intersect]
    test.all_x = test.all_x[intersect]
    predictions, report, matrix = decicionTest(train.all_x, train.all_y, test.all_x, test.all_y)
    score = f1_score_1(report)
    return score, [train, test, predictions, report, matrix] 
#     return score

In [470]:
def create_train_test(train_start_day, test_start_day, m_size, tar):

#     建立物件
    train = X(train_start_day, m_size) 
    #     combine multiple test
    tests = []
    for i in range(test_start_day,(test_start_day-30),(m_size*(-1))):
        test = X(i,m_size)
        tests.append(test)
        
    train.attach()
    for test in tests:
        test.attach()
        
#     選擇目標
    if tar=='t':
        train.generate_1(train.t)
        for test in tests:
            test.generate_1(test.t)
    elif tar=='c':
        train.generate_1(train.c)
        for test in tests:
            test.generate_1(test.c)
    elif tar=='f':
        train.generate_1(train.f)
        for test in tests:
            test.generate_1(test.f)
    elif tar=='l':
        train.generate_1(train.l)
        for test in tests:
            test.generate_1(test.l)
            
#     篩選交集
    d = [list(i.all.columns) for i in [train]+tests]
    intersect = list(set(d[0]).intersection(*d))
    train.all = train.all[intersect]
    for i in tests:
        i.all = i.all[intersect]
    train.all = train.all.reindex(sorted(train.all.columns), axis=1)
    for i in tests:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)

    train.all['y'] = train.all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    train.all_x = train.all[list(train.all.columns[4:-1])]
    train.all_y = train.all['y']
    
    test_all = pd.concat([i.all for i in tests]).groupby('CUST_NO').max().reset_index()
    test_all['y'] = test_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    test_all_x = test_all[list(test_all.columns[4:-1])]
    test_all_y = test_all['y'] 

    return train, test_all, test_all_x, test_all_y

In [471]:
def Create_train_test(train_start_day, test_start_day, m_size, tar):

#     建立物件
    #     combine multiple train
    trains = []
    for i in range(train_start_day,(train_start_day-30),(m_size*(-1))):
        train = X(i,m_size)
        trains.append(train)
  
    #     combine multiple test
    tests = []
    for i in range(test_start_day,(test_start_day-30),(m_size*(-1))):
        test = X(i,m_size)
        tests.append(test)
        
    for train in trains:
        train.attach()
    for test in tests:
        test.attach()
        
#     選擇目標
    if tar=='t':
        for train in trains:
            train.generate_1(train.t)
        for test in tests:
            test.generate_1(test.t)
    elif tar=='c':
        for train in trains:
            train.generate_1(train.c)
        for test in tests:
            test.generate_1(test.c)
    elif tar=='f':
        for train in trains:
            train.generate_1(train.f)
        for test in tests:
            test.generate_1(test.f)
    elif tar=='l':
        for train in trains:
            train.generate_1(train.l)
        for test in tests:
            test.generate_1(test.l)
            
#     篩選交集
    d = [list(i.all.columns) for i in trains+tests]
    intersect = list(set(d[0]).intersection(*d))
    
    for i in trains:
        i.all = i.all[intersect]
    for i in tests:
        i.all = i.all[intersect]
        
    for i in trains:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)
    for i in tests:
        i.all = i.all.reindex(sorted(i.all.columns), axis=1)

    train_all = pd.concat([i.all for i in trains]).groupby('CUST_NO').max().reset_index()
    train_all['y'] = train_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    train_all_x = train_all[list(train_all.columns[4:-1])]
    train_all_y = train_all['y']
    
    test_all = pd.concat([i.all for i in tests]).groupby('CUST_NO').max().reset_index()
    test_all['y'] = test_all['NEW_TXN_DT_Count'].apply(lambda x:1 if x>0 else 0)
    test_all_x = test_all[list(test_all.columns[4:-1])]
    test_all_y = test_all['y'] 

    return train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y

In [472]:
def experiment_score(train_all_x, train_all_y, test_all_x, test_all_y):
    predictions, report, matrix = decicionTest(train_all_x, train_all_y, test_all_x, test_all_y)
    score = f1_score_1(report)
    return score, [predictions, report, matrix]

In [476]:
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 30, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.11
c  score: 0.04
f  score: 0.41
l  score: 0.02
train: (92858, 55), test: (86911, 54)


In [477]:
for i in ['t', 'c', 'f', 'l']:
    score, ref = x_comparing_score(31, 61, 5, i)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(ref[0].all.shape, ref[1].all.shape))

t  score: 0.17
c  score: 0.01
f  score: 0.5
l  score: 0.07
train: (22630, 49), test: (24133, 48)


In [483]:
for i in ['t', 'c', 'f', 'l']:
    train, test_all, test_all_x, test_all_y = create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train.all_x, train.all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

t  score: 0.17
c  score: 0.02
f  score: 0.43
l  score: 0.1
train: (92858, 42), test: (86911, 43)


In [484]:
for i in ['t', 'c', 'f', 'l']:
    train_all, train_all_x, train_all_y, test_all, test_all_x, test_all_y = Create_train_test(31, 61, 5, i)
    score, ref = experiment_score(train_all_x, train_all_y, test_all_x, test_all_y)
    print('{}  score: {}'.format(i, score))
    if i=='l':
        print('train: {}, test: {}'.format(train_all.shape, test_all.shape))

t  score: 0.12
c  score: 0.04
f  score: 0.42
l  score: 0.13
train: (92858, 42), test: (86911, 42)


In [449]:
score

0.43

In [59]:
for i in ['t', 'c', 'f', 'l']:
    score = x_comparing_score(31, 61, 7, i)[0]
    print('cuurent item: {}'.format(i))
    print('score: {}'.format(score))

cuurent item: t
score: 0.14
cuurent item: c
score: 0.04
cuurent item: f
score: 0.47
cuurent item: l
score: 0.07


In [79]:
print()

# score_collection

In [72]:
def score_collection(train_start_day, test_start_day, tar):
    x_score = {}
    print('m_size:', end=' ')
    for m in range(1,15):
        x_score[m] = x_comparing_score(train_start_day=train_start_day, 
                                       test_start_day=test_start_day, m_size=m, tar=tar)[0]
        print(m, end=' ')
    return x_score

In [78]:
c_score = score_collection(train_start_day=31, test_start_day=61, tar='c')

m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

In [80]:
c_score

{1: 0.02,
 2: 0.01,
 3: 0.03,
 4: 0.02,
 5: 0.0,
 6: 0.0,
 7: 0.04,
 8: 0.04,
 9: 0.05,
 10: 0.04,
 11: 0.04,
 12: 0.04,
 13: 0.04,
 14: 0.03}

In [61]:
f_record = {}
for train_start_day in range(61,30,-1):
    print('train_start_day: {}'.format(train_start_day), end='  ')
    f_score = {}
    for m in range(1,15):
        f_score[m] = comparing_score(train_start_day=train_start_day, test_start_day=91, m_size=m)[0]
        print(m, end=' ')    
    f_record[train_start_day] = f_score
    print('\n')

train_start_day: 61
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 60
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 59
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 58
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 57
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 56
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 55
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 54
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 53
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 52
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 51
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 50
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 49
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 48
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 47
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 46
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 45
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 44
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_star

In [63]:
max_f_record = {}
for k in list(f_record.keys()):
    max_f_record[k] = max_score_day(f_record[k])

In [69]:
f_s = [i[0] for i in list(max_f_record.values())]
f_d = [i[1] for i in list(max_f_record.values())]
avg_f_s = sum(f_s) / len(f_s)
avg_f_d = sum(f_d) / len(f_d)

In [70]:
print(avg_f_s, avg_f_d)

0.17709677419354838 3.978494623655914


In [73]:
c_record = {}
for train_start_day in range(61,30,-1):
    print('train_start_day: {}'.format(train_start_day), end='  ')
    c_score = score_collection(train_start_day=train_start_day, test_start_day=91, tar='c')   
    c_record[train_start_day] = c_score

train_start_day: 61  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 60  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 59  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 58  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 57  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 56  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 55  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 54  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 53  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 52  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 51  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 50  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 49  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 48  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 47  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 

train_start_day: 46  m_size: 1 2 3 4 5 6

In [81]:
def X_record(tar):
    x_record = {}
    for train_start_day in range(61,30,-1):
        print('train_start_day: {}'.format(train_start_day), end='  ')
        x_score = score_collection(train_start_day=train_start_day, test_start_day=91, tar=tar)   
        x_record[train_start_day] = x_score
        print()
    return x_record

In [82]:
def max_record(x_record):
    max_x_record = {}
    for k in list(x_record.keys()):
        max_x_record[k] = max_score_day(x_record[k])
    x_s = [i[0] for i in list(max_x_record.values())]
    x_d = [i[1] for i in list(max_x_record.values())]
    avg_x_s = sum(x_s) / len(x_s)
    avg_x_d = sum(x_d) / len(x_d)
    return avg_x_s, avg_x_d

In [76]:
max_record(c_record)

(0.05516129032258067, 7.181720430107528)

In [83]:
f_record = X_record(tar='f')
print('\n')
print('max_score: {}, max_date: {}'.format(max_record(f_record))

train_start_day: 61  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 60  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 59  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 58  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 57  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 56  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 55  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 54  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 53  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 52  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 51  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 50  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 49  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 48  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 47  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 46  m_size: 1 2 3 4 5 6 7 8 9 10 11 12

IndexError: tuple index out of range

In [88]:
max_record(f_record)

(0.5003225806451612, 1.096774193548387)

In [89]:
max_f = max_record(f_record)

max_score: 0.5003225806451612, max_date: 1.096774193548387


In [91]:
l_record = X_record(tar='l')

train_start_day: 61  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 60  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 59  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 58  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 57  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 56  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 55  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 54  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 53  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 52  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 51  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 50  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 49  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 48  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 47  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 46  m_size: 1 2 3 4 5 6 7 8 9 10 11 12

In [92]:
t_record = X_record(tar='t')

train_start_day: 61  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 60  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 59  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 58  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 57  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 56  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 55  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 54  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 53  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 52  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 51  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 50  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 49  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 48  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 47  m_size: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 
train_start_day: 46  m_size: 1 2 3 4 5 6 7 8 9 10 11 12

In [131]:
for i in [t_record, c_record, f_record, l_record]:
    max_i = max_record(i)
    print('max_score: {}, max_date: {}'.format(max_i[0], max_i[1]))

max_score: 0.1770967741935484, max_date: 4.072580645161291
max_score: 0.05516129032258067, max_date: 7.181720430107528
max_score: 0.5003225806451612, max_date: 1.096774193548387
max_score: 0.0964516129032258, max_date: 4.196236559139784


In [127]:
for i in [t_record, c_record, f_record, l_record]:
    print(i[31])

{1: 0.12, 2: 0.13, 3: 0.13, 4: 0.15, 5: 0.15, 6: 0.15, 7: 0.14, 8: 0.15, 9: 0.14, 10: 0.14, 11: 0.14, 12: 0.13, 13: 0.12, 14: 0.11}
{1: 0.04, 2: 0.01, 3: 0.03, 4: 0.03, 5: 0.01, 6: 0.03, 7: 0.05, 8: 0.04, 9: 0.05, 10: 0.04, 11: 0.04, 12: 0.05, 13: 0.04, 14: 0.05}
{1: 0.5, 2: 0.43, 3: 0.45, 4: 0.43, 5: 0.42, 6: 0.39, 7: 0.4, 8: 0.39, 9: 0.37, 10: 0.37, 11: 0.37, 12: 0.36, 13: 0.36, 14: 0.34}
{1: 0.03, 2: 0.08, 3: 0.04, 4: 0.07, 5: 0.08, 6: 0.07, 7: 0.09, 8: 0.07, 9: 0.06, 10: 0.06, 11: 0.05, 12: 0.04, 13: 0.03, 14: 0.03}


# Test

In [143]:
score, ref = x_comparing_score(train_start_day=61, 
                  test_start_day=91, m_size=1, tar='f')

In [144]:
score

0.51

In [148]:
ref[0].all.shape

(8877, 42)

In [149]:
ref[1].all.shape

(6169, 42)

In [156]:
score, ref = x_comparing_score(train_start_day=91, 
                  test_start_day=121, m_size=1, tar='f')
print(score, ref[0].all.shape, ref[1].all.shape)

0.0 (6169, 42) (2975, 40)


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [159]:
score, ref = x_comparing_score(train_start_day=91, 
                  test_start_day=121, m_size=30, tar='f')
print(score, ref[0].all.shape, ref[1].all.shape)

0.0 (86867, 54) (30000, 55)


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# combine

In [108]:
for i,k in [('t',1), ('c',2), ('f',3), ('l',5)]:
    print(i,k, end='   ')

t 1   c 2   f 3   l 5   

In [133]:
final_tar = {}
final_prd = {}
print('cuurent item:', end=' ')
for i,k in [('t',4), ('c',7), ('f',1), ('l',7)]:
    train, test, predictions, report, matrix = x_comparing_score(train_start_day=61, 
                                                                 test_start_day=121, m_size=k, tar=i)[1]
    print(i, end=' ')
    final_tar[i] = test.all
    final_prd[i] = predictions

cuurent item: 

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


t c f l 

In [167]:
final_tar['c'].shape

(11673, 45)

In [169]:
final_prd['c'].shape

(11673,)

In [100]:
final_res = {}
for seq in ['t', 'c', 'f', 'l']:
    if seq=='t':
        final_tar[seq]['wm_prediction'] = final_prd[seq]
        final_res[seq] = final_tar[seq][['CUST_NO', 'wm_prediction']]
    elif seq=='c':
        final_tar[seq]['cc_prediction'] = final_prd[seq]
        final_res[seq] = final_tar[seq][['CUST_NO', 'cc_prediction']]
    elif seq=='f':
        final_tar[seq]['fx_prediction'] = final_prd[seq]
        final_res[seq] = final_tar[seq][['CUST_NO', 'fx_prediction']]
    elif seq=='l':
        final_tar[seq]['ln_prediction'] = final_prd[seq]
        final_res[seq] = final_tar[seq][['CUST_NO', 'ln_prediction']]

In [101]:
zero = pd.read_csv('Downloads/dataset/TBN_Y_ZERO.csv')
zero.shape

(30000, 5)

In [170]:
final_res['c'].shape

(11673, 2)

In [102]:
from functools import reduce
d = [zero, final_res['c'], final_res['f'], final_res['l'], final_res['t']]
result = reduce(lambda left,right: pd.merge(left,right,how='left',on='CUST_NO'), d)

In [171]:
final_res['c'].head()

,CUST_NO,cc_prediction
0,--JVR0JBCXHAROP8,0
1,--NQCBUYH8AY5GJW,0
2,--OR0BOBUBUTKZFC,0
3,-03MWQSKSBMMT6XC,0
4,-07A3DDW-8RW7ASQ,0


In [563]:
result.drop(['CC_IND', 'FX_IND', 'LN_IND', 'WM_IND'], axis=1, inplace=True)
result.rename(columns = {'cc_prediction': 'CC_IND', 
                         'fx_prediction': 'FX_IND', 
                         'ln_prediction': 'LN_IND', 
                         'wm_prediction': 'WM_IND', }, inplace = True)

In [ ]:
result.head()

In [565]:
result.to_csv('result.csv',index=True)